In [ ]:
!pip install ultralytics


In [ ]:
!pip install flask pyngrok opencv-python pillow torch torchvision \
    git+https://github.com/openai/CLIP.git \
    git+https://github.com/facebookresearch/segment-anything.git \
    tf2onnx


In [ ]:
!mkdir -p anomaly_ui/templates
!mkdir -p anomaly_ui/static
!mkdir -p anomaly_ui/uploads


In [ ]:
%%writefile anomaly_ui/templates/index.html
<!DOCTYPE html>
<html>
<head>
<title>Industrial Anomaly Detection</title>

<style>
body {
    font-family: Segoe UI, Arial, sans-serif;
    background: #f4f6f9;
    margin: 0;
}

.header {
    background: #111827;
    color: white;
    padding: 16px;
    text-align: center;
    font-size: 22px;
}

.container {
    max-width: 1200px;
    margin: 30px auto;
    background: white;
    padding: 25px;
    border-radius: 8px;
    box-shadow: 0 12px 28px rgba(0,0,0,0.12);
}

.upload-box {
    border: 2px dashed #cbd5e1;
    padding: 30px;
    text-align: center;
    border-radius: 6px;
}

.file-name {
    margin-top: 10px;
    color: #2563eb;
    font-weight: 500;
}

button {
    background: #2563eb;
    color: white;
    border: none;
    padding: 12px 24px;
    border-radius: 6px;
    font-size: 15px;
    cursor: pointer;
    margin-top: 15px;
}

button:disabled {
    background: #94a3b8;
    cursor: not-allowed;
}

.results {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: 25px;
    margin-top: 25px;
}

.card {
    border: 1px solid #e5e7eb;
    padding: 18px;
    border-radius: 6px;
}

.defect { color: #dc2626; font-weight: bold; }
.normal { color: #16a34a; font-weight: bold; }

img {
    width: 100%;
    border-radius: 6px;
    border: 1px solid #e5e7eb;
}
</style>
</head>

<body>

<div class="header">
YOLO + SAM + CLIP — Industrial Anomaly Detection
</div>

<div class="container">

<form method="POST" enctype="multipart/form-data" onsubmit="return validateForm()">
    <div class="upload-box">
        <h3>Select Product Image</h3>

        <input type="file" id="imageInput" name="image" accept="image/*" required>

        <div class="file-name" id="fileName">No file selected</div>

        <button id="analyzeBtn" type="submit" disabled>Analyze Image</button>
    </div>
</form>

{% if score is not none %}
<div class="results">

    <div class="card">
        <h3>Detection Result</h3>
        <p>Status:
            <span class="{{ 'defect' if status=='DEFECT' else 'normal' }}">
                {{ status }}
            </span>
        </p>
        <p>Anomaly Score: <b>{{ score }}</b></p>
    </div>

    <div class="card">
        <h3>Segmentation Result</h3>
        <img src="{{ url_for('static', filename='after.png') }}">
    </div>

</div>
{% endif %}

</div>

<script>
const input = document.getElementById("imageInput");
const fileName = document.getElementById("fileName");
const button = document.getElementById("analyzeBtn");

input.addEventListener("change", () => {
    if (input.files.length > 0) {
        fileName.innerText = input.files[0].name;
        button.disabled = false;
    } else {
        fileName.innerText = "No file selected";
        button.disabled = true;
    }
});

function validateForm() {
    if (input.files.length === 0) {
        alert("Please select an image before analyzing.");
        return false;
    }
    return true;
}
</script>

</body>
</html>


In [ ]:
%%writefile anomaly_ui/app.py
import os
import cv2
import torch
import clip
import numpy as np
from flask import Flask, render_template, request
from werkzeug.utils import secure_filename
from segment_anything import sam_model_registry, SamPredictor
from ultralytics import YOLO

# =====================================================
# PATHS & CONFIG
# =====================================================
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
UPLOAD_FOLDER = os.path.join(BASE_DIR, "uploads")
STATIC_FOLDER = os.path.join(BASE_DIR, "static")
TEMPLATE_FOLDER = os.path.join(BASE_DIR, "templates")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(STATIC_FOLDER, exist_ok=True)

# =====================================================
# FLASK APP
# =====================================================
app = Flask(
    __name__,
    static_folder=STATIC_FOLDER,
    template_folder=TEMPLATE_FOLDER
)
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

# =====================================================
# LOAD MODELS
# =====================================================

# ---- YOLO (for box prompt)
yolo = YOLO("yolov8l.pt")

# ---- CLIP
clip_model, _ = clip.load("ViT-L/14", device=DEVICE)
clip_model.eval()

# ---- SAM
SAM_CKPT = os.path.join(BASE_DIR, "sam_vit_h_4b8939.pth")
if not os.path.exists(SAM_CKPT):
    import urllib.request
    urllib.request.urlretrieve(
        "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth",
        SAM_CKPT
    )

sam = sam_model_registry["vit_h"](checkpoint=SAM_CKPT)
sam.to(DEVICE)
sam.eval()
sam_predictor = SamPredictor(sam)

# =====================================================
# UTILS
# =====================================================
def get_yolo_box(img):
    results = yolo.predict(img, conf=0.3, verbose=False)
    if len(results[0].boxes) == 0:
        return None
    box = results[0].boxes[0].xyxy[0].cpu().numpy()
    return box.astype(int)

def smooth_mask(mask):
    kernel = np.ones((7,7), np.uint8)
    mask = mask.astype(np.uint8) * 255
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    return mask > 0

def refine_edges(mask):
    edges = cv2.Canny(mask.astype(np.uint8)*255, 100, 200)
    return edges

def overlay_mask(img, mask, color=(255,0,0)):
    overlay = img.copy()
    overlay[mask] = color
    return overlay

# =====================================================
# CLIP FEATURE
# =====================================================
def extract_clip_feature(img):
    img = cv2.resize(img, (224,224))
    img = img.astype(np.float32)/255.0
    mean = np.array([0.48145466, 0.4578275, 0.40821073])
    std  = np.array([0.26862954, 0.26130258, 0.27577711])
    img = (img - mean) / std
    img = np.transpose(img, (2,0,1))
    tensor = torch.tensor(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        feat = clip_model.encode_image(tensor)
        feat = feat / feat.norm(dim=-1, keepdim=True)
    return feat

# =====================================================
# PATCH EXTRACTION
# =====================================================
def extract_patches(img, mask, size=64, stride=32):
    patches = []
    h, w, _ = img.shape
    for y in range(0, h-size, stride):
        for x in range(0, w-size, stride):
            if mask[y:y+size, x:x+size].mean() < 0.05:
                continue
            patches.append(img[y:y+size, x:x+size])
    if not patches:
        patches.append(img[h//2-size:h//2+size, w//2-size:w//2+size])
    return patches

# =====================================================
# ROUTE
# =====================================================
@app.route("/", methods=["GET","POST"])
def index():
    score, status = None, None

    if request.method == "POST":
        file = request.files["image"]
        filename = secure_filename(file.filename)
        path = os.path.join(UPLOAD_FOLDER, filename)
        file.save(path)

        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # ---------- BEFORE (Unguided SAM)
        sam_predictor.set_image(img)
        raw_masks, _, _ = sam_predictor.predict(multimask_output=True)
        raw_mask = max(raw_masks, key=lambda x: x.sum())

        # ---------- YOLO-GUIDED SAM
        box = get_yolo_box(img)
        if box is not None:
            masks, _, _ = sam_predictor.predict(
                box=box,
                multimask_output=False
            )
            mask = masks[0]
        else:
            mask = raw_mask

        # ---------- SMOOTH + REFINE
        smooth = smooth_mask(mask)
        edges = refine_edges(smooth)

        # ---------- VISUALS
        before = overlay_mask(img, raw_mask)
        after = overlay_mask(img, smooth)

        cv2.imwrite(
            os.path.join(STATIC_FOLDER, "before.png"),
            cv2.cvtColor(before, cv2.COLOR_RGB2BGR)
        )
        cv2.imwrite(
            os.path.join(STATIC_FOLDER, "after.png"),
            cv2.cvtColor(after, cv2.COLOR_RGB2BGR)
        )

        # ---------- ANOMALY SCORE
        patches = extract_patches(img, smooth)
        sims = [extract_clip_feature(p).mean().item() for p in patches]
        score = round(float(np.mean(sims)), 3)
        status = "DEFECT" if score > 1.8 else "NORMAL"

    return render_template("index.html", score=score, status=status)

# =====================================================
# MAIN
# =====================================================
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=False)


In [ ]:
# ===============================
# 6️⃣ Kill any previous processes
# ===============================
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"




In [ ]:
!lsof -i :5000 || echo "Port 5000 is free"

In [ ]:
!kill -9 8882

In [ ]:
# ===============================
# 7️⃣ Run Flask in the background
# ===============================
!nohup python anomaly_ui/app.py > flask.log 2>&1 &


In [ ]:
# ===============================
# 8️⃣ Start ngrok tunnel
# ===============================
from pyngrok import ngrok, conf

conf.get_default().auth_token = "replace your token"

public_url = ngrok.connect(5000)
print("🌍 Public URL:", public_url)


In [ ]:
# ===============================
# 9️⃣ Check logs
# ===============================
!sleep 3 && tail -n 20 flask.log
